# Installing Dependencies

In [1]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python 
!pip mediapipe sklearn matplotlib install pyttsx3

You should consider upgrading via the 'C:\Users\Muzamil Khattak\AppData\Local\Programs\Python\Python37\python.exe -m pip install --upgrade pip' command.


In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import pyttsx3

In [2]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [3]:
def draw_landmarks(image, results):
    mp_drwaing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) #Draw Face Connections
    mp_drwaing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)  #Draw Pose Connections
    mp_drwaing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) #Draw Left Hand Connections
    mp_drwaing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) #Draw Right Hand Connections

In [4]:
def draw_styled_landmarks(image,results):
    mp_drwaing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                              mp_drwaing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                              mp_drwaing.DrawingSpec(color=(80, 256, 121), thickness=1,  circle_radius=1)
                              )
    # Draw Face Connections
    mp_drwaing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drwaing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                              mp_drwaing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1)

                             # mp_drwaing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                             # mp_drwaing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2)
                              )  # Draw Pose Connections
    mp_drwaing.draw_landmarks(image, results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                              mp_drwaing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                              mp_drwaing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
                              )  # Draw Left Hand Connections
    mp_drwaing.draw_landmarks(image, results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                              mp_drwaing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                              mp_drwaing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                              )  # Draw Right Hand Connections


# Keypoints using MP Holistic

In [5]:
mp_holistic = mp.solutions.holistic   #Holistoc model
mp_drwaing = mp.solutions.drawing_utils  #drwaing utilies

In [6]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # read feeds
       ret, frame = cap.read()

       #Make Detection
       image, results = mediapipe_detection(frame, holistic)
       print(results)

       #Draw Landmarks
       draw_styled_landmarks(image,results)

       #Show to Screen
       cv2.imshow('My Cam ', image)

        # Quit loop
       if cv2.waitKey(10) & 0xFF == ord('q'):
        break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [7]:
len(results.face_landmarks.landmark)

468

In [8]:
draw_styled_landmarks(frame,results)

In [1]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

NameError: name 'plt' is not defined

In [9]:
len(results.pose_landmarks.landmark)

33

# Extracting Keypoint Values

In [10]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [11]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [12]:
face

array([ 0.35335445,  0.65731359, -0.01531924, ...,  0.43092191,
        0.51783854,  0.00785819])

In [13]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [14]:
extract_keypoints(results).shape

(1662,)

# Setup Folders for Collection

In [15]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['Add','Agree','Approve','Argue','Arrive','Attack','Bite','Boil','Bond','Call',
                    'Cancel','Confrim','Delay','Destroy', 'Die','Draw','Drink','Dance','Deal','Dress', 
                    'Explain','Easy','Fan','Fall','Fight','Fix','Find','Fly','Gather','Hug','Hello' , 
                    'howAreYou' ,'Help','Hit','Hope','Injured','I','Irritate','Insult' ,'Kiss','Knock',
                    'Know','Learn','Laugh','Live','Like','Listen','Lie','Love','Make','Match','Need','Now',
                    'Power','Stop','Thanks','Try','Want','You'])
#'Hello' ,'Thanks', 'howAreYou', 'Fan' ,'Want','Add','Agree'
#'Insult','Love','Now','Try','Hit','Hope','Injured','Help','Easy'
#'Insult','Love','Now','Try','Hit','Hope','Injured','Help','Easy'
#'cancel','confrim','delay','Destroy', 'Die','draw',Drink,'Dance','Fall'
#'Dress', 'Power','Explain','Fight','Fix','Find','Fly''Gather','Irritate'
#'Hug','Kiss','Knock','Know','Learn','Laugh'
#'Live','Like','Listen','Lie','Make','Match','Need','Stop'
# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 30

In [16]:
for action in actions: 
     for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# Collect Keypoint Value for Training and Testing

In [84]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    Sign_no= 0
    for action in actions:
        x = input("press Start Key to Countinue FOR  the sign " + actions[Sign_no] + " ")
        Sign_no+=1
        # Loop through sequences aka videos
        for sequence in range(no_sequences):

            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
                print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

press Start Key to Countinue FOR  the sign I 
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.Sol

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

...

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [31]:
# Actions that we try to detect
actions = np.array(['Add','Agree','Approve','Argue','Arrive','Attack','Bite','Boil','Bond','Call','Cancel','Confrim','Delay','Destroy', 'Die','Draw','Drink','Dance','Deal','Dress', 'Explain','Easy','Fan','Fall','Fight','Fix','Find','Fly','Gather','Hug','Hello' , 'howAreYou' ,'Help','Hit','Hope','Injured','I','Irritate','Insult' ,'Kiss','Knock','Know','Learn','Laugh','Live','Like','Listen','Lie','Love','Make','Match','Need','Now', 'Power','Stop','Thanks','Try','Want','You'])
print(actions)

['Add' 'Agree' 'Approve' 'Argue' 'Arrive' 'Attack' 'Bite' 'Boil' 'Bond'
 'Call' 'Cancel' 'Confrim' 'Delay' 'Destroy' 'Die' 'Draw' 'Drink' 'Dance'
 'Deal' 'Dress' 'Explain' 'Easy' 'Fan' 'Fall' 'Fight' 'Fix' 'Find' 'Fly'
 'Gather' 'Hug' 'Hello' 'howAreYou' 'Help' 'Hit' 'Hope' 'Injured' 'I'
 'Irritate' 'Insult' 'Kiss' 'Knock' 'Know' 'Learn' 'Laugh' 'Live' 'Like'
 'Listen' 'Lie' 'Love' 'Make' 'Match' 'Need' 'Now' 'Power' 'Stop' 'Thanks'
 'Try' 'Want' 'You']


# Preprocess Data and Create Labels and Features

In [17]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [18]:
label_map = {label:num for num, label in enumerate(actions)}

In [19]:
label_map

{'Add': 0,
 'Agree': 1,
 'Approve': 2,
 'Argue': 3,
 'Arrive': 4,
 'Attack': 5,
 'Bite': 6,
 'Boil': 7,
 'Bond': 8,
 'Call': 9,
 'Cancel': 10,
 'Confrim': 11,
 'Delay': 12,
 'Destroy': 13,
 'Die': 14,
 'Draw': 15,
 'Drink': 16,
 'Dance': 17,
 'Deal': 18,
 'Dress': 19,
 'Explain': 20,
 'Easy': 21,
 'Fan': 22,
 'Fall': 23,
 'Fight': 24,
 'Fix': 25,
 'Find': 26,
 'Fly': 27,
 'Gather': 28,
 'Hug': 29,
 'Hello': 30,
 'howAreYou': 31,
 'Help': 32,
 'Hit': 33,
 'Hope': 34,
 'Injured': 35,
 'I': 36,
 'Irritate': 37,
 'Insult': 38,
 'Kiss': 39,
 'Knock': 40,
 'Know': 41,
 'Learn': 42,
 'Laugh': 43,
 'Live': 44,
 'Like': 45,
 'Listen': 46,
 'Lie': 47,
 'Love': 48,
 'Make': 49,
 'Match': 50,
 'Need': 51,
 'Now': 52,
 'Power': 53,
 'Stop': 54,
 'Thanks': 55,
 'Try': 56,
 'Want': 57,
 'You': 58}

In [20]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
#     print(labels)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,

...

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,

In [21]:
np.array(sequences).shape

(1770, 30, 1662)

In [22]:
X = np.array(sequences)

In [23]:
X.shape

(1770, 30, 1662)

In [24]:
y =to_categorical(labels).astype(int)

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [28]:
X_train.shape

(1768, 30, 1662)

# Build and Train LSTM Neural Network

In [1]:
from tensorflow.keras.models import Sequential
# from tensorflow import set_random_seed
# from numpy.random import seed
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard
import random
random.seed(2)
np.random.seed(1)
tf.random.set_seed(2)




# import tensorflow as tf
# tf.keras.utils.set_random_seed(2)
# np.random.seed(1)



In [59]:
log_dir = os.path.join('Logs4')
tb_callback = TensorBoard(log_dir=log_dir)

In [37]:
model = Sequential()
model.add(LSTM(32, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=False, activation='relu'))

model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
# model.add(Dropout(0.2))
# model.add(LSTM(64, return_sequences=True, activation='relu'))
#

# model.add(Dropout(0.5))
# model.add(Dense(64, activation='relu'))
#model.add(Dense(32, activation='relu'))

In [38]:
X.shape

(1770, 30, 1662)

In [39]:
model.compile(optimizer='Adam', loss='',
              metrics=categorical_crossentropy['categorical_accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 30, 32)            216960    
_________________________________________________________________
lstm_7 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dense_3 (Dense)              (None, 59)                3835      
Total params: 245,627
Trainable params: 245,627
Non-trainable params: 0
_________________________________________________________________


In [40]:
model.fit(X_train, y_train, epochs=100, callbacks=[tb_callback])

Epoch 1/100
56/56 [==============================] - 4s 41ms/step - loss: 4.6838 - categorical_accuracy: 0.0200
Epoch 2/100
56/56 [==============================] - 2s 35ms/step - loss: 4.0914 - categorical_accuracy: 0.0325
Epoch 3/100
56/56 [==============================] - 2s 35ms/step - loss: 4.0084 - categorical_accuracy: 0.0275
Epoch 4/100
56/56 [==============================] - 2s 35ms/step - loss: 3.7648 - categorical_accuracy: 0.0536
Epoch 5/100
56/56 [==============================] - 2s 35ms/step - loss: 3.3835 - categorical_accuracy: 0.0832
Epoch 6/100
56/56 [==============================] - 2s 35ms/step - loss: 3.1407 - categorical_accuracy: 0.1185
Epoch 7/100
56/56 [==============================] - 2s 35ms/step - loss: 2.9043 - categorical_accuracy: 0.1810
Epoch 8/100
56/56 [==============================] - 2s 36ms/step - loss: 2.5419 - categorical_accuracy: 0.2398
Epoch 9/100
56/56 [==============================] - 2s 35ms/step - loss: 2.3233 - categorical_accuracy:

In [62]:
model = Sequential()
model.add(LSTM(32, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(Dropout(0.2))
# model.add(LSTM(64, return_sequences=True, activation='relu'))
# model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=False, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(64, activation='relu'))
#model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

        
        
        
EPOCHS = 100
checkpoint_filepath = 'MP_data'

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,save_weights_only=True,monitor='val_accuracy',mode='max',save_best_only=True)
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

# Model weights are saved at the end of every epoch, if it's the best seen
# so far.
model.fit(X_train, y_train, epochs=EPOCHS, callbacks=[model_checkpoint_callback])


Epoch 1/100
56/56 [==============================] - 7s 43ms/step - loss: 4.1043 - categorical_accuracy: 0.0220
Epoch 2/100
56/56 [==============================] - 2s 43ms/step - loss: 4.0288 - categorical_accuracy: 0.0405
Epoch 3/100
56/56 [==============================] - 2s 42ms/step - loss: 3.8042 - categorical_accuracy: 0.0558
Epoch 4/100
56/56 [==============================] - 2s 38ms/step - loss: 3.5047 - categorical_accuracy: 0.0553
Epoch 5/100
56/56 [==============================] - 2s 38ms/step - loss: 3.3374 - categorical_accuracy: 0.0838
Epoch 6/100
56/56 [==============================] - 2s 41ms/step - loss: 3.3189 - categorical_accuracy: 0.0743
Epoch 7/100
56/56 [==============================] - 2s 40ms/step - loss: 3.1586 - categorical_accuracy: 0.0846
Epoch 8/100
56/56 [==============================] - 2s 40ms/step - loss: 3.0225 - categorical_accuracy: 0.1234
Epoch 9/100
56/56 [==============================] - 3s 47ms/step - loss: 2.9920 - categorical_accuracy:

Epoch 43/100
56/56 [==============================] - 2s 39ms/step - loss: 1.7898 - categorical_accuracy: 0.4072
Epoch 44/100
56/56 [==============================] - 2s 40ms/step - loss: 1.7732 - categorical_accuracy: 0.4067
Epoch 45/100
56/56 [==============================] - 2s 40ms/step - loss: 1.7952 - categorical_accuracy: 0.4078
Epoch 46/100
56/56 [==============================] - 2s 40ms/step - loss: 1.8129 - categorical_accuracy: 0.3970
Epoch 47/100
56/56 [==============================] - 2s 41ms/step - loss: 1.7669 - categorical_accuracy: 0.4118
Epoch 48/100
56/56 [==============================] - 2s 40ms/step - loss: 1.7868 - categorical_accuracy: 0.4344
Epoch 49/100
56/56 [==============================] - 2s 41ms/step - loss: 1.6957 - categorical_accuracy: 0.4312
Epoch 50/100
56/56 [==============================] - 2s 40ms/step - loss: 1.7377 - categorical_accuracy: 0.4167
Epoch 51/100
56/56 [==============================] - 2s 41ms/step - loss: 1.6083 - categorical_

Epoch 85/100
56/56 [==============================] - 2s 42ms/step - loss: 1.3743 - categorical_accuracy: 0.5398
Epoch 86/100
56/56 [==============================] - 2s 41ms/step - loss: 1.3874 - categorical_accuracy: 0.5272
Epoch 87/100
56/56 [==============================] - 2s 41ms/step - loss: 1.3187 - categorical_accuracy: 0.5539
Epoch 88/100
56/56 [==============================] - 2s 40ms/step - loss: 1.4351 - categorical_accuracy: 0.5112
Epoch 89/100
56/56 [==============================] - 2s 40ms/step - loss: 1.3498 - categorical_accuracy: 0.5381
Epoch 90/100
56/56 [==============================] - 2s 40ms/step - loss: 1.3895 - categorical_accuracy: 0.5310
Epoch 91/100
56/56 [==============================] - 2s 40ms/step - loss: 1.3377 - categorical_accuracy: 0.5462
Epoch 92/100
56/56 [==============================] - 2s 40ms/step - loss: 1.3092 - categorical_accuracy: 0.5398
Epoch 93/100
56/56 [==============================] - 2s 41ms/step - loss: 1.3869 - categorical_

In [63]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 30, 32)            216960    
_________________________________________________________________
dropout_2 (Dropout)          (None, 30, 32)            0         
_________________________________________________________________
lstm_13 (LSTM)               (None, 64)                24832     
_________________________________________________________________
dense_6 (Dense)              (None, 59)                3835      
Total params: 245,627
Trainable params: 245,627
Non-trainable params: 0
_________________________________________________________________


In [42]:
# from tensorflow import keras
# model = keras.models.load_model('action2.h5')

OSError: SavedModel file does not exist at: action2.h5/{saved_model.pbtxt|saved_model.pb}

# Make Predictions

In [47]:
res = model.predict(X_test)

In [50]:
actions[np.argmax(res[1])]

'Laugh'

In [52]:
actions[np.argmax(y_test[0])]

'Make'

# Save weights

In [46]:
model.save('action2.h5')

# Evaluation using Confusion Matrix and Accuracy

In [53]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [54]:
yhat = model.predict(X_train)

In [55]:
ytrue= np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis = 1).tolist()

In [56]:
multilabel_confusion_matrix(ytrue,yhat)

array([[[1738,    0],
        [  14,   16]],

       [[1738,    0],
        [  12,   18]],

       [[1738,    0],
        [  10,   20]],

       [[1733,    5],
        [  10,   20]],

       [[1725,   13],
        [   8,   22]],

       [[1738,    0],
        [  30,    0]],

       [[1733,    5],
        [   1,   29]],

       [[1736,    2],
        [   2,   28]],

       [[1735,    3],
        [  14,   16]],

       [[1731,    7],
        [   9,   21]],

       [[1738,    0],
        [   9,   21]],

       [[1735,    3],
        [  11,   19]],

       [[1738,    0],
        [  14,   16]],

       [[1729,    9],
        [   6,   24]],

       [[1738,    0],
        [   1,   29]],

       [[1736,    2],
        [   0,   30]],

       [[1738,    0],
        [  14,   16]],

       [[1738,    0],
        [   5,   25]],

       [[1727,   11],
        [   1,   29]],

       [[1738,    0],
        [   0,   30]],

       [[1738,    0],
        [  14,   16]],

       [[1732,    6],
        [   

In [58]:
accuracy_score(ytrue, yhat)

0.8059954751131222

# Test in Real Time

In [46]:
from scipy import stats

In [47]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [245]:
# load_weights('action.h5')

NameError: name 'load_weights' is not defined

In [48]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res,actions,image,colors))

IndexError: list index out of range

<Figure size 1296x1296 with 0 Axes>

In [52]:
!pip install pyttsx3

You should consider upgrading via the 'C:\Users\Muzamil Khattak\AppData\Local\Programs\Python\Python37\python.exe -m pip install --upgrade pip' command.


In [39]:
# initialize Text-to-speech engine
engine = pyttsx3.init()
# convert this text to speech
engine.say(sentence)
# play the speech
engine.runAndWait()

In [36]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
        #3. Viz logic
#             if res[np.argmax(res)] > threshold:
            if np.unique(predictions[-10:])[0]==np.argmax(res):
                if res[np.argmax(res)] > threshold:
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

#             # Viz probabilities
#             image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        
        
        # convert this text to speech
#         text = "Python is a great programming language"
#         engine.say(sentence)
#         # play the speech
#         engine.runAndWait()



        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Deal
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Deal
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Deal
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Deal
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Deal
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Deal
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Deal
<class 'mediapipe.python.solution_base.Solutio

Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe

Listen
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Listen
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Listen
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Approve
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Approve
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Approve
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Approve
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Approve
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Approve
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Approve
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Approve
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Approve
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Approve
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Approve
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Approve
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Approve
<class 'mediapipe.py

Thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thanks
<class 'mediapipe.python.solution

...

Fly
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Fly
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Fly
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Fly
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Fly
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Fly
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Fly
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Fly
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Fly
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Fly
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Fly
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Fly
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Fly
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Fly
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Fly
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Fly
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Fly
<class 'mediapipe.py

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Fly
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Fly
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Fly
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Fly
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Fly
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Fly
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Fly
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Fly
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Fly
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Fly
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Fly
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Fly
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Fly
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Fly
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Fly
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thanks
<class 'mediapipe.pyt

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Dance
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Dance
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Dance
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.

Arrive
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Arrive
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Arrive
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Arrive
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Arrive
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Arrive
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Arrive
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Arrive


In [40]:
# convert this text to speech
engine.say(sentence)
# play the speech
engine.runAndWait()
